In [1]:
import sys
sys.path.append('C://Users/olive/OneDrive - Australian National University/Honours-Olivia/Programs/honours/standard_modules')
sys.path.append('C://Users/olive/OneDrive - Australian National University/Honours-Olivia/Programs/honours/project')

from collation import collator, hvc_snapshot as snap, collation_tools as ct
from synthesis import hvc_looper as hvcl, magnetic_field_derivation as mfd, KStest, uncertainty_subtraction as us

#collated_data = collator.data_whole_sky(False, load_data=["../data_processed/proc_rms","../data_processed/proc_hvcs"], h1_img="../data_catalog/hi4pi-hvc-nhi-car.fits", override_RMs=True)

master_hvcs = hvcl.load_HVCs([], [], "../data_processed/toy_model/toy_hvcs/", toy_override=True)

results = ct.read_processed("../data_processed/toy_model/wgm_results")

len(master_hvcs)

=== HVC RM LOADER ===
Taking HVC snapshots
Process complete


3072

In [2]:
import numpy as np

In [4]:
master_rm_inner, master_rm_outer, inners, outers = us.get_stacked_sets(master_hvcs, toy_override=True)
inner_sigma = us.uncertainty_table(inners)
outer_sigma = us.uncertainty_table(outers)

#uncertainty_KS = np.sqrt(inner_sigma["Sigma [meas]"]**2 + inner_sigma["Sigma [obsv]"]**2 + outer_sigma["Sigma [meas]"]**2 + outer_sigma["Sigma [obsv]"]**2)
uncertainty_KS = np.sqrt(inner_sigma["Sigma [meas]"]**2 + outer_sigma["Sigma [meas]"]**2)

uncertainty_KS

C:\/Users/olive/OneDrive - Australian National University/Honours-Olivia/Programs/honours/project\synthesis.py:429: RuntimeWarning: invalid value encountered in sqrt
  uncert.append({"Sigma [meas]":meas, "Sigma [obsv]":obsv, "Sigma [true]": np.sqrt(obsv**2 - meas**2)})
C:\/Users/olive/OneDrive - Australian National University/Honours-Olivia/Programs/honours/project\synthesis.py:429: RuntimeWarning: invalid value encountered in sqrt
  uncert.append({"Sigma [meas]":meas, "Sigma [obsv]":obsv, "Sigma [true]": np.sqrt(obsv**2 - meas**2)})


0.8910528192480175
0.8375052614630897
0.7636317655746471
0.8180791567349067
0.875084546819616
0.8184384611938141
0.7636317655746472
0.838343727070891
0.8579183898190748
0.8128309868069494
0.7802959932617584


In [5]:
results.add_column(uncertainty_KS, name="KS unc")

results

Name,Statistic,p_value,Statistic_x,Statistic_sgn,Statistic_diff,Significant,Sigma [meas],Sigma [obsv],Sigma [true],Sigma [diff],FWHM,Average [inner],Average [outer],Average [diff],Avg Unc [inner],Avg Unc [outer],Avg Unc [diff],KS unc
str67,float64,float64,float64,int32,float64,bool,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
toy_0_0.0_0.0_0.0.ecsv,0.15512615945713734,0.02416490935579006,0.546875,-1,-0.0036755505505539077,False,0.7181247448032667,0.43199068054162376,nan,nan,nan,0.13206216582583086,0.06187903704760876,0.0701831287782221,0.0019822617659745454,0.0001830740741053514,0.001990697823742734,0.8910528192480175
toy_0_0.0_0.0_0.7853981633974483.ecsv,0.15512615945713734,0.02416490935579006,0.546875,-1,-0.0036755505505539077,False,0.6504944929722889,0.2787985633411827,nan,nan,nan,0.1413938042103961,0.06187903704760876,0.07951476716278734,0.00205210872803258,0.0001830740741053514,0.0020602588061399045,0.8375052614630897
toy_0_0.0_0.0_1.5707963267948966.ecsv,0.12254738650909154,0.1250838416045315,0.109375,-1,0.059324949949946415,False,0.5521426408126524,0.20198135638324727,nan,nan,nan,0.09215149775079459,0.06187903704760876,0.03027246070318583,0.0011376728117382048,0.0001830740741053514,0.0011523087881196794,0.7636317655746471
toy_0_0.0_0.0_2.356194490192345.ecsv,0.0917395802282969,0.4127193027413549,0.3203125,1,-0.13013795045045384,False,0.6252851582222411,0.2553787524570038,nan,nan,nan,0.13185791111151304,0.06187903704760876,0.06997887406390428,0.0020832703328715514,0.0001830740741053514,0.002091298973468948,0.8180791567349067
toy_0_0.0_0.0_3.141592653589793.ecsv,0.10641580894601183,0.2439601275322874,-0.2327370731741204,1,-0.18268702312407392,False,0.6982127086292612,0.4046139519692637,nan,nan,nan,0.1266502050670992,0.06187903704760876,0.06477116801949044,0.002114574047548032,0.0001830740741053514,0.0021224842800768155,0.875084546819616
toy_0_0.0_0.0_3.9269908169872414.ecsv,0.11772544756360576,0.15422986809584666,0.36328125,1,-0.08716920045045384,False,0.6257551735065523,0.2577992608363467,nan,nan,nan,0.1321422854170526,0.06187903704760876,0.07026324836944385,0.0020904068600567567,0.0001830740741053514,0.002098408196033813,0.8184384611938141
toy_0_0.0_0.0_4.71238898038469.ecsv,0.12254738650909154,0.1250838416045315,0.109375,-1,0.059324949949946415,False,0.5521426408126525,0.20198135638324727,nan,nan,nan,0.09215149775079559,0.06187903704760876,0.03027246070318683,0.0011376728117382176,0.0001830740741053514,0.001152308788119692,0.7636317655746472
toy_0_0.0_0.0_5.497787143782138.ecsv,0.1644719538496607,0.014050395172507146,0.546875,-1,-0.0036755505505539077,False,0.6515736544152392,0.2760786238074689,nan,nan,nan,0.1426047883545859,0.06187903704760876,0.08072575130697714,0.0020562409259283283,0.0001830740741053514,0.002064374690329283,0.838343727070891
toy_0_0.0_0.7853981633974483_0.0.ecsv,0.12708877627956724,0.10164890612991566,0.546875,-1,-0.0036755505505539077,False,0.6765737106913561,0.35500403496551103,nan,nan,nan,0.14009990953013815,0.06187903704760876,0.07822087248252939,0.0021563223549002325,0.0001830740741053514,0.0021640799927109943,0.8579183898190748


In [6]:
ct.write_processed(results, "../data_processed/toy_model/results_final_unfiltered")
ct.write_processed(us.filter_significant(results), "../data_processed/toy_model/results_final_filtered")

## Bootstrap Resampling

In [5]:
from astropy.table import Table
import copy

def bootstrap_selection(sample):
    sample = copy.deepcopy(sample)
    sample.remove_column("ra_dec_obj")
    rand = np.round((len(sample)-1) * np.random.rand(len(sample))).astype(int)
    t = copy.deepcopy(sample)
    t.remove_rows(list(range(len(sample))))

    for index in rand:
        t.add_row(list(sample[index]))
    
    return t

def bootstrap_sample_creation(sample, console_out=""):
    samples = []
    l = len(sample)
    for i in range(len(sample)):
        samples.append(bootstrap_selection(sample))
        print(console_out+"Creating samples: "+str(int((i+1)/l*100))+"% \r", sep="", end="", flush=True)
    return samples

def bootstrap_evaluation(samples, callback):
    sample_out = []
    l = len(samples)

    for i in range(len(samples)):
        sout = "Evaluating samples: "+str(int((i+1)/l*100))+"% "
        sample = samples[i]
        bootstrapped = bootstrap_sample_creation(sample, sout+"")
        response = np.array(list(map(callback, bootstrapped)))
        sample_out.append(response)
        print(sout+"\r", sep="", end="", flush=True)

    return np.array(sample_out)

def uncertainty_calculate(rms):
    m_list = rms["RM_uncert"].data
    o_list = rms["RM"].data

    meas = np.mean(m_list)
    obsv = np.std(o_list)

    return np.sqrt(obsv**2 - meas**2)

In [19]:
import warnings

def bootstrap_uncertaintes(master_hvcs):
    with warnings.catch_warnings(action="ignore"):
        print("=== BOOTSTRAPPING UNCERTAINTIES ===")
        print("Getting stacked sets")
        master_rm_inner, master_rm_outer, inners, outers = us.get_stacked_sets(master_hvcs)
        print("Calculating inner magnetic field uncertainties")
        inner_mag = bootstrap_evaluation(inners, uncertainty_calculate)
        inner_unc = []
        for l in inner_mag:
            inner_unc.append(np.nanstd(l))
        inner_unc = np.array(inner_unc)
        print() 
        print("Calculating outer magnetic field uncertainties")
        outer_mag = bootstrap_evaluation(outers, uncertainty_calculate)
        outer_unc = []
        for l in outer_mag:
            outer_unc.append(np.nanstd(l))
        outer_unc = np.array(outer_unc)
        print()
        print("Calculating final uncertainty")
        final = np.sqrt(inner_unc ** 2 + outer_unc ** 2)
        return final

field_set = bootstrap_uncertaintes(master_hvcs)
field_set

=== BOOTSTRAPPING UNCERTAINTIES ===
Getting stacked sets
Calculating inner magnetic field uncertainties
Calculating outer magnetic field uncertainties% 
Calculating final uncertaintyting samples: 100% 


array([0.39584818, 0.32473521, 0.20428799, 1.2244449 ,        nan,
       0.85192194, 1.51154246, 8.32530501, 1.63155893, 1.66239263,
       0.30293187, 1.12654023, 0.46018009, 0.65763306, 0.38222939])

In [20]:
results.add_column(field_set, name="Sigma unc")

results

Name,Statistic,p_value,Statistic_x,Statistic_sgn,Statistic_diff,Significant,Sigma [meas],Sigma [obsv],Sigma [true],Sigma [diff],FWHM,Average [inner],Average [outer],Average [diff],Avg Unc [inner],Avg Unc [outer],Avg Unc [diff],KS unc,Sigma unc
str15,float64,float64,float64,int32,float64,bool,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
G038.7-37.3-241,0.2074738871132492,3.5194997048780594e-09,-3.515687526708854,-1,2.5503785393572116,True,1.5490103064419392,3.259633374868092,2.8680615072014404,-1.2004037875138898,-2.82673490096878,-1.3506194176296213,-2.560210571743284,1.2095911541136628,0.007003852876365774,0.0032940466469477043,0.007739812557552317,2.1815601020063724,0.39584818265424776
G038.8-33.2-261,0.31066825775656326,1.3943532792837091e-08,-0.6963302660734451,-1,1.245611675868496,True,1.0172162310775188,2.2736826533710723,2.033446322742001,-0.3820916381721009,-0.8997570486063758,-0.056445222198439816,-1.1991611171284744,1.1427158949300344,0.010148842996047043,0.0006398107274504492,0.010168990703377291,1.4553771530792659,0.3247352108401835
G043.4-30.4-230,0.24109469074986317,3.606854288675973e-11,-1.2589038059025925,-1,2.2846397376409495,True,1.3692498776485982,3.4139391636197307,3.1273207998949974,-0.3219533860094135,-0.7581422869405536,1.500776660653911,0.052073639648805965,1.448703021005105,0.007129552203987939,0.002050088534124988,0.007418448464951416,2.014296014920469,0.20428798707875953
G089.0-64.7-311,0.3738140417457306,0.015075834341180974,-3.5031459148744912,1,-2.4420848538134305,False,1.7766805216998156,6.285240753776755,6.028901861595415,4.354124583319416,10.253179847362592,-2.0433500968670546,-0.7515102964160036,-1.291839800451051,0.18157752049511933,0.007404423938031661,0.18172842772392428,2.4332628530195217,1.2244448962835837
G133.5-75.6-294,0.46130268199233715,4.570648166268751e-08,-1.0081759072724916,-1,1.7746068755102902,True,1.390653120203451,0.6443423969361269,nan,nan,nan,-0.3570725165898588,-1.1535705764067303,0.7964980598168715,0.042906373222247746,0.007109176796687973,0.0434913469303192,2.001602544846478,nan
G248.9+36.8+181,0.08070447918650822,0.452664079053446,-2.5040967925547646,-1,0.5189262304682574,False,2.5577313774843344,3.3456971641453612,2.1567799412092095,-0.4370240961740395,-1.0291131018321615,-0.7672410917916938,-0.7426862708275596,-0.024554820964134253,0.04655645277823943,0.01318809477546906,0.04838831614242422,3.6192905843246095,0.8519219357184207
G261.5-34.5+237,0.2889454915944982,5.960798790546759e-08,-5.487954609053061,-1,7.064597943499491,True,8.363070544185819,12.507923833010205,9.300925205879048,5.8850039182248,13.858125191721438,-3.620108937244275,-7.837450117500442,4.217341180256167,0.4686915287204748,0.13939402347543306,0.4889810250664184,11.798096482827903,1.5115424552737302
G262.4-26.2+143,0.21329644533869885,1.2224926075128765e-06,2.0255699650566523,-1,5.008552948039634,True,5.0791565705667105,9.744382180123484,8.315957708164293,-11.277167064796421,-26.555699055345446,4.0206400164297795,0.17353602031256268,3.8471039961172173,0.11708586076209412,0.0424759648413749,0.12455242422211704,7.193251686060157,8.325305008340273
G264.6-53.3+224,0.3195887445887446,4.0730287570954555e-05,0.9062634882158793,1,-2.1167595348071426,True,3.023390368905306,4.983729795251348,3.9619027435421326,0.16014564072012671,0.377114164892079,0.8210758855823943,2.499888949334954,-1.67881306375256,0.16176219741916012,0.013152603813846201,0.1622960242919072,4.269997743503358,1.6315589302445745


In [21]:
ct.write_processed(results, "../data_processed/results_final_unfiltered")
ct.write_processed(us.filter_significant(results), "../data_processed/results_final_filtered")

# KS Uncertainty Formalisation

In [ ]:
import sys
sys.path.append('C://Users/olive/OneDrive - Australian National University/Honours-Olivia/Programs/honours/standard_modules')
sys.path.append('C://Users/olive/OneDrive - Australian National University/Honours-Olivia/Programs/honours/project')

from collation import collator, hvc_snapshot as snap, collation_tools as ct
from synthesis import hvc_looper as hvcl, magnetic_field_derivation as mfd, KStest, uncertainty_subtraction as us
